In [6]:
from langchain_core.messages import HumanMessage, AIMessage, RemoveMessage, ToolMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()


True

In [ ]:

# --- Conversation Simulation ---

# 1️⃣ Regular conversation (no tool call)
human_1 = HumanMessage(content="Hey there! How are you today?")
ai_1 = AIMessage(content="I'm doing great! How can I assist you today?")

# 2️⃣ Human asks something factual (no tool call)
human_2 = HumanMessage(content="Can you tell me who won the 2022 FIFA World Cup?")
ai_2 = AIMessage(content="Argentina won the 2022 FIFA World Cup, defeating France in the final.")

# 3️⃣ Human asks something requiring a tool call
human_3 = HumanMessage(content="What's the current weather in New York?")
ai_3 = AIMessage(
    content="",
    tool_calls=[
        {
            "id": "tool_call_1",
            "name": "get_weather",
            "args": {"location": "New York"}
        }
    ]
)
tool_1 = ToolMessage(
    content='{"temperature": "12°C", "condition": "Cloudy"}',
    tool_call_id="tool_call_1"
)

# 4️⃣ AI responds after getting tool result
ai_4 = AIMessage(content="The current weather in New York is 12°C and cloudy. Would you like a forecast for tomorrow?")

# 5️⃣ Continue normal conversation (no tool call)
human_4 = HumanMessage(content="No, that's fine. Can you remind me what I asked earlier?")
ai_5 = AIMessage(content="You first asked who won the 2022 FIFA World Cup, and then about the weather in New York.")

# --- Combine all messages ---
messages = [
    human_1, ai_1,
    human_2, ai_2,
    human_3, ai_3, tool_1
    , ai_4,
    human_4, ai_5
]

In [ ]:
for message in messages:
    print(f"{type(message)}: {message.content}")

<class 'langchain_core.messages.human.HumanMessage'>: Hey there! How are you today?
<class 'langchain_core.messages.ai.AIMessage'>: I'm doing great! How can I assist you today?
<class 'langchain_core.messages.human.HumanMessage'>: Can you tell me who won the 2022 FIFA World Cup?
<class 'langchain_core.messages.ai.AIMessage'>: Argentina won the 2022 FIFA World Cup, defeating France in the final.
<class 'langchain_core.messages.human.HumanMessage'>: What's the current weather in New York?
<class 'langchain_core.messages.ai.AIMessage'>: 
<class 'langchain_core.messages.tool.ToolMessage'>: {"temperature": "12°C", "condition": "Cloudy"}
<class 'langchain_core.messages.ai.AIMessage'>: The current weather in New York is 12°C and cloudy. Would you like a forecast for tomorrow?
<class 'langchain_core.messages.human.HumanMessage'>: No, that's fine. Can you remind me what I asked earlier?
<class 'langchain_core.messages.ai.AIMessage'>: You first asked who won the 2022 FIFA World Cup, and then abo

In [14]:

# --- Step 2: Initialize the OpenAI chat model ---
llm = ChatOpenAI(
    model="gpt-5-nano-2025-08-07",  # or gpt-4o, gpt-4-turbo, etc.
    temperature=0
)

# --- Step 3: Continue the conversation with the existing messages ---
response = llm.invoke(messages)

# --- Step 4: Print the model’s continuation ---
print(response.content)

Right now in New York: 12°C and cloudy. If you’re stepping out, a light jacket would be comfortable. Want an hourly forecast or details like wind or humidity?


In [39]:
last_human_message = None
last_human_message_index = None

for i, message in enumerate(messages):
    if isinstance(message, HumanMessage):
        last_human_message = message
        last_human_message_index = i

print(last_human_message_index, last_human_message.content)

8 No, that's fine. Can you remind me what I asked earlier?


In [ ]:
current_conversation = messages[last_human_message_index:]
for message in current_conversation:
    print(f"{type(message)}: {message.content}")

<class 'langchain_core.messages.human.HumanMessage'>: No, that's fine. Can you remind me what I asked earlier?
<class 'langchain_core.messages.ai.AIMessage'>: You first asked who won the 2022 FIFA World Cup, and then about the weather in New York.


In [43]:
MAX_HISTORY_LENGTH = 2
COUNT = 0
recent_history = []

# extract anything before last human_human_message_index till max_history_length considering the only the human messages will be covered when calculating the MAX_HISTORY_LENGTH
for index, message in reversed(list(enumerate(messages[:last_human_message_index]))):
    
    if isinstance(message, HumanMessage):
        recent_history.append(message)
        COUNT += 1
        if COUNT == MAX_HISTORY_LENGTH:
            break
    else:
        recent_history.append(message)

recent_history = reversed(recent_history)



for message in recent_history:
    print(f"{type(message)}: {message.content}")

<class 'langchain_core.messages.human.HumanMessage'>: Can you tell me who won the 2022 FIFA World Cup?
<class 'langchain_core.messages.ai.AIMessage'>: Argentina won the 2022 FIFA World Cup, defeating France in the final.
<class 'langchain_core.messages.human.HumanMessage'>: What's the current weather in New York?
<class 'langchain_core.messages.ai.AIMessage'>: 
<class 'langchain_core.messages.tool.ToolMessage'>: {"temperature": "12°C", "condition": "Cloudy"}
<class 'langchain_core.messages.ai.AIMessage'>: The current weather in New York is 12°C and cloudy. Would you like a forecast for tomorrow?
